## Setup
Importing libraries and connecting to tmdb api:

---

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [2]:
## Loading api credentials from file on local machine
with open("C:\\Users\\josep\\.secret\\imdbapi.json", 'r') as f:
    login = json.load(f)

login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY =  login['api-key']

In [4]:
movie = tmdb.Movies(603)
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/l4QHerTSbMI7qgvasqxP36pqjN6.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 84.369,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

## First Function
Creating function to add rating to movie information for only US movies.

---

In [5]:
## Creating function to add rating
def get_movie_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    
    for c in releases["countries"]:
        if c["iso_3166_1"] == "US":
            movie_info["certifcation"] = c["certification"]
        return movie_info

In [6]:
## Testing Function
test = get_movie_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 178.888,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

## Creating Second Function
---

In [7]:
## Creating variables
folder = "Data/"
years = [2000, 2001]
basics = pd.read_csv("Data/basics.cvs.gz")

In [8]:
def write_json(new_data, filename):
    
    with open(filename, "r") as file:
        file_data = json.load(file)
        
        if(type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else: 
            file_data.append(new_data)
        
        file.seek(0)
        
        json.dump(file_data, file)

In [9]:
for Year in tqdm_notebook(years, desc = "years", position = 0):
    ## Defining JSON file to store year results
    JSON_FILE = f"{folder}tmdb.api_results_{Year}.json"
    
    ## Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## Creating file if it does not exist
    if file_exists == False:
        
        with open(JSON_FILE, "w") as f:
            json.dump([{"imdb_id" : 0}], f)
    
    ## Creating new dataframe based on year
    df = basics.loc[basics["startYear"] == Year].copy()
    
    ## Saving movie ids to list
    movie_ids = df["tconst"].copy()
    
    ## Loading existing data to filter
    previous_df = pd.read_json(JSON_FILE)
    
    ## Filter movies already in file
    movie_ids_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]
    
    for movie_id in tqdm_notebook(movie_ids_get, desc=f'Movies from {Year}', position = 1, leave = True):
    
        try: 
            temp = get_movie_rating(movie_id)
            
            write_json(temp, JSON_FILE)
            
            time.sleep(0.02)
        
        except Exception as e:
            continue 
    
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'{folder}final_tmdb_data{Year}.csv.gz', compression = "gzip", index = False)

years:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1439 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1562 [00:00<?, ?it/s]

In [10]:
df_2000 = pd.read_csv("Data/final_tmdb_data2000.csv.gz")
df_2000.head()

,imdb_id
0,0


In [11]:
test = pd.read_csv("Data/")

PermissionError: [Errno 13] Permission denied: 'Data/'